In [ ]:
import urllib
import urllib.request as ur
import numpy
import csv

def parseJSONData(fname):
    with open(fname) as f:
        content = f.readlines()
    for l in content:
        yield eval(l)

In [ ]:
def getYs(JSON):
    values = []
    for i in range(len(JSON)):
        if(JSON[i]["helpful"]["outOf"] != 0):
            values.append(JSON[i]["helpful"]["nHelpful"]/JSON[i]["helpful"]["outOf"])
        #else:
            #values.append(0)
    return values
        

In [ ]:
data = list(parseJSONData("C:/Users/Kyle/HelpfulnessData/train.json"))

In [ ]:
data[0]

In [ ]:
Training = data[:100000]
Validation = data[100000:200000]

In [ ]:
ytrain = getYs(Training)
print(len(ytrain))

In [ ]:
alpha = numpy.average(ytrain)
print(alpha)

#### ANSWER

Alpha = 0.783737364232

In [ ]:
yvalid = getYs(Validation)
dif = numpy.average(numpy.abs(yvalid - alpha))
print(dif)

#### ANSWER

MAE = 0.277603731206

In [ ]:
def features(JSON, alpha):
    values = numpy.zeros((100000, 3))
    count = 0
    for i in range(len(JSON)):
        if(JSON[i]["helpful"]["outOf"] != 0):
            values[count][0] = 1
            values[count][1] = len(JSON[i]["reviewText"].split())
            values[count][2] = JSON[i]["rating"]
            count += 1
    return values    

In [ ]:
Xtrain = features(Training, alpha)[:31402]
# print(Xtrain)
# print(numpy.shape(Xtrain))

In [ ]:
ytrain = getYs(Training)

In [ ]:
theta,residuals,rank,s = numpy.linalg.lstsq(Xtrain, ytrain)

print(theta)


Yvalid = getYs(Validation)
Xvalid = features(Validation, alpha)[:len(Yvalid)]

In [ ]:
prediction = numpy.dot(Xvalid,theta)

In [ ]:
mae = numpy.average(numpy.abs(prediction - Yvalid))

In [ ]:
print(mae)

#### ANSWER

MAE = 0.269304779898

In [ ]:
reviewData = list(parseJSONData("C:/Users/Kyle/HelpfulnessData/test_Helpful.json"))
reviewData[8649]

In [ ]:
print(len(reviewData))

In [ ]:
def readCSV(fname):
    with open(fname) as f:
        reader = list(csv.reader(f,delimiter = '-'))
    return reader

In [ ]:
#userID-itemID-outOf,prediction
pairs = readCSV("C:/Users/Kyle/HelpfulnessData/pairs_Helpful.txt")
print(len(pairs))

In [ ]:
def featurestest(JSON, alpha):
    values = numpy.zeros((14000, 3))
    count = 0
    for i in range(len(JSON)):
        values[count][0] = 1
        values[count][1] = len(JSON[i]["reviewText"].split())
        values[count][2] = JSON[i]["rating"]
        count += 1
    return values   

xtest = featurestest(reviewData, alpha)
print(xtest[0])
print(numpy.shape(xtest))

In [ ]:
rightside = numpy.dot(xtest, theta)
print (rightside[0])

In [ ]:
outof = numpy.array(pairs)[:,2]
print("hi", outof[0])
outof2 = outof.astype(int)
predictions = numpy.multiply(rightside, outof2)
print (predictions[0])

In [ ]:
file = open("C:/Users/Kyle/HelpfulnessData/pairs_Helpful_mine.txt", 'w')
file.write("userID-itemID-outOf,prediction\r")
for i in range(len(predictions)):
    file.write(str(reviewData[i]["reviewerID"]) + "-" + str(reviewData[i]["itemID"]) + "-" + str(reviewData[i]["helpful"]["outOf"]) + "," + str(predictions[i]) + "\r")

In [ ]:
ratings = numpy.zeros(100000)
for i in range(len(Training)):
        ratings[i] = Training[i]["rating"]
ratingAlpha = numpy.average(ratings)
print(ratingAlpha)

validratings = numpy.zeros(100000)
for i in range(len(Training)):
    validratings[i] = Validation[i]["rating"]

dif = numpy.average(numpy.abs(validratings - ratingAlpha))
print(dif)

#### ANSWER

Alpha = 4.23198    
MAE = 0.8894957272

In [ ]:
def updateAlpha(userdata, itemdata, alpha, n):
    sum = 0
    for user in userdata:
        for item in userdata[user]['items']:
            sum += int(userdata[user]['items'][item]) - (userdata[user]['userBias'] + itemdata[item]['itemBias'])
    sum = sum / n
    return sum
    
def updateUser(userdata, itemdata, alpha, lam):
    for user in userdata:
        sum = 0
        for item in userdata[user]['items']:
            sum += int(userdata[user]['items'][item]) - (alpha + itemdata[item]['itemBias'])
        sum = sum / (lam + len(userdata[user]['items']))
        userdata[user]['userBias'] = sum
    
def updateItem(userdata, itemdata, alpha, lam):
    for item in itemdata:
        sum = 0
        for user in itemdata[item]['users']:
            sum += int(itemdata[item]['users'][user]) - (alpha + userdata[user]['userBias'])
        sum = sum / (lam + len(itemdata[item]['users']))
        itemdata[item]['itemBias'] = sum    
        
        
        
def updateAll(userdata, itemdata, alpha, n, lam, eps):
    dif = 1
    while(dif > eps):
        alphaold = alpha
        alpha = updateAlpha(userdata, itemdata, alpha, n)
        updateUser(userdata, itemdata, alpha, lam)
        updateItem(userdata, itemdata, alpha, lam)
        dif = numpy.abs(alpha - alphaold)
        #print(alpha)
    return alpha

In [ ]:
def getUserData(Training):
    userdata = {}
    for element in Training:
        key = element["reviewerID"]
        if(key not in userdata):
            userdata[key] = {
                'userBias' : 0, 
                'items' : {
                    element['itemID'] : element['rating']
                }
            }
        else:
            userdata[key]['items'][element['itemID']] = element['rating']
    return userdata

In [ ]:
def getItemData(Training):
    itemdata = {}
    for element in Training:
        key = element["itemID"]
        if(key not in itemdata):
            itemdata[key] = {
                'itemBias' : 0, 
                'users' : {
                    element['reviewerID'] : element['rating']
                }
            }
        else:
            itemdata[key]['users'][element['reviewerID']] = element['rating']
    return itemdata


In [ ]:
min = 0
max = 0
username = ""
usernamemin = ""
for user in userdata:
    if(userdata[user]['userBias'] < min):
        min = userdata[user]['userBias']
        username = user
    if(userdata[user]['userBias'] > max):
        max = userdata[user]['userBias']
        usernamemin = user
print(username, " ", usernamemin)
print(max, " ", min)
    

In [ ]:
min = 0
max = 0
itemname = ""
itemnamemin = ""
for item in itemdata:
    if(itemdata[item]['itemBias'] < min):
        min = itemdata[item]['itemBias']
        itemname = item
    if(itemdata[item]['itemBias'] > max):
        max = itemdata[item]['itemBias']
        itemnamemin = item
print(itemname, " ", itemnamemin)
print(max, " ", min)

#### ANSWER

Min User: U052814411    
Max User: U816486110    
    
Min Item: I071368828    
Max Item: I558325415

In [ ]:
def getMSE(alpha, userdata, itemdata, data):
    sum = 0
    for i in range(len(data)):
        user = 0
        item = 0
        if(data[i]['reviewerID'] in userdata):
            user = userdata[data[i]['reviewerID']]['userBias']
        if(data[i]['itemID'] in itemdata):
            item = itemdata[data[i]['itemID']]['itemBias']
        sum += ((alpha + user + item - data[i]['rating'])**2)
    sum = sum / len(data)
    return sum

In [ ]:
alpha = updateAll(userdata, itemdata, 1, len(Training) , 2.5, .0001)
print("MSE = ", getMSE(alpha, userdata, itemdata, data))

In [ ]:
tests = [.01, .1, 1, 2, 3, 6, 7, 10, 100]
for i in tests:
    userdata = getUserData(Training)
    itemdata = getItemData(Training)
    alpha = updateAll(userdata, itemdata, 1, len(Training) , i, .0001)
    print("MSE @ ", i, " = ", getMSE(alpha, userdata, itemdata, Validation))

In [ ]:
userdata = getUserData(Training)
itemdata = getItemData(Training)
alpha = updateAll(userdata, itemdata, 1, len(Training) , 7, .0001)
print("MSE @ ", 7, " = ", getMSE(alpha, userdata, itemdata, Validation))

In [ ]:
#userID-itemID,prediction
pairs = readCSV("C:/Users/Kyle/HelpfulnessData/pairs_Rating.txt")
print(numpy.shape(pairs))

In [ ]:
def getPredict(alpha, userdata, itemdata):
    pairs = readCSV("C:/Users/Kyle/HelpfulnessData/pairs_Rating.txt")
    print(numpy.shape(pairs))
    print(alpha)
    prediction = numpy.zeros(14000)
    for i in range(len(pairs)):
        user = 0
        item = 0
        if(pairs[i][0] in userdata):
            user = userdata[pairs[i][0]]['userBias']
        if(pairs[i][1] in itemdata):
            item = itemdata[pairs[i][1]]['itemBias']
        prediction[i] = (alpha + user + item)
    return prediction

In [ ]:
file = open("C:/Users/Kyle/HelpfulnessData/pairs_Rating_mine.txt", 'w')
file.write("userID-itemID,prediction\r")
predic = getPredict(alpha, userdata, itemdata)


In [ ]:
for i in range(len(pairs)):
    file.write(str(pairs[i][0]) + "-" + str(pairs[i][1]) + "," + str(predic[i]) + "\r")
    print(i)

#### ANSWER

Lambda Chosen: 7     
MSE @  7  =  1.1396040704237156    
